In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
import json

In [3]:
from series_dict import series_dict_1, series_dict_2, series_dict_3, series_dict

Need to Initialize variables for the token and url values

In [4]:
token = '70e5b01b221e44558b7ea4b6bf6ba0be'
url = 'https://api.bls.gov/publicAPI/v2/timeseries'

In [5]:
df = pd.DataFrame()

Creating the key per BLS specifications

In [6]:
key = '?registrationkey={}'.format(token)

The API call to gather the json data happens here

In [7]:
headers = {'Content-type': 'application/json'}

# Range of years to pull, we're only looking at 2023
dates = ('2023', '2023')

data1 = json.dumps({
    "seriesid": list(series_dict_1.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

data2 = json.dumps({
    "seriesid": list(series_dict_2.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

data3 = json.dumps({
    "seriesid": list(series_dict_3.keys()),
    "startyear": dates[0],
    "endyear": dates[1]})

response1 = requests.post(
    '{}{}'.format(f'{url}/data/', key),
    headers=headers,
    data=data1).json()['Results']['series']

response2 = requests.post(
    '{}{}'.format(f'{url}/data/', key),
    headers=headers,
    data=data2).json()['Results']['series']

response3 = requests.post(
    '{}{}'.format(f'{url}/data/', key),
    headers=headers,
    data=data3).json()['Results']['series']


In [8]:
# Sample json data
print(response3)

[{'seriesID': 'LASST400000000000003', 'data': [{'year': '2023', 'period': 'M12', 'periodName': 'December', 'value': '3.5', 'footnotes': [{}]}, {'year': '2023', 'period': 'M11', 'periodName': 'November', 'value': '3.5', 'footnotes': [{}]}, {'year': '2023', 'period': 'M10', 'periodName': 'October', 'value': '3.5', 'footnotes': [{}]}, {'year': '2023', 'period': 'M09', 'periodName': 'September', 'value': '3.5', 'footnotes': [{}]}, {'year': '2023', 'period': 'M08', 'periodName': 'August', 'value': '3.3', 'footnotes': [{}]}, {'year': '2023', 'period': 'M07', 'periodName': 'July', 'value': '3.2', 'footnotes': [{}]}, {'year': '2023', 'period': 'M06', 'periodName': 'June', 'value': '3.1', 'footnotes': [{}]}, {'year': '2023', 'period': 'M05', 'periodName': 'May', 'value': '3.0', 'footnotes': [{}]}, {'year': '2023', 'period': 'M04', 'periodName': 'April', 'value': '2.9', 'footnotes': [{}]}, {'year': '2023', 'period': 'M03', 'periodName': 'March', 'value': '3.0', 'footnotes': [{}]}, {'year': '2023

Defining function to programmatically pull the nessesary data out of json for use in a dataframe

In [9]:
# This function pulls the values out of the json data and finds the mean per seriesID

def parse_json_response(responses):

  df = pd.DataFrame()

  # Initialze blank list for the values and the seriesIDs
  unemployed_column_list = list()
  states_column_list = list()

  # Iterate through each item in the response
  for i in responses:
    states_column_list.append(i['seriesID'])
    unemploy_value = list()

    # Iterate through each set of data in the response
    for j in i['data']:
      unemploy_value.append(float(j['value']))

    unemployed_column_list.append(np.mean(unemploy_value))

  df['Unemployment Rate'] = unemployed_column_list
  df['State'] = states_column_list

  return df

In [10]:
df1 = parse_json_response(response1)
df2 = parse_json_response(response2)
df3 = parse_json_response(response3)

In [11]:
df = pd.concat([df1, df2, df3], ignore_index=True)

In [12]:
# Use the series dictionary to rename the states
df['State'] = df['State'].replace(series_dict)

Final unemployment dataframe includes a column for the unemployment average for 2023 and the state

In [13]:
df.head(50)

,Unemployment Rate,State
0,2.491667,Alabama
1,4.216667,Alaska
2,3.908333,Arizona
3,3.225000,Arkansas
4,4.741667,California
5,3.150000,Colorado
6,3.708333,Connecticut
7,3.983333,Delaware
8,2.908333,Florida
9,3.200000,Georgia


In [14]:
df.to_csv('unemployment_df.csv')